Thie notebook explores the multilayer perceptron for binary text classification for your text classification problem, using the keras library.  Before starting, be sure to install the following versions of tensorflow and keras (for python 3.7):

```sh
pip install tensorflow==1.13.0-rc2
pip install keras==2.2.4
```

In [ ]:
import keras
import numpy as np
from sklearn import preprocessing
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from random import choices
import pandas as pd

In [ ]:
def read_data(filename):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for idx,line in enumerate(file):
            cols=line.rstrip().split("\t")
            label=cols[0]
            text=cols[1]
            X.append(text)
            Y.append(label)

    return X, Y

In [ ]:
# Change this to the directory with your data (from the CheckData_TODO.ipynb exercise).  
# The directory should contain train.tsv, dev.tsv and test.tsv
directory="../data/text_classification_sample_data"

In [ ]:
trainX, trainY=read_data("%s/train.tsv" % directory)
devX, devY=read_data("%s/dev.tsv" % directory)
testX, testY=read_data("%s/test.tsv" % directory)

vectorizer = CountVectorizer(max_features=10000, analyzer=str.split, lowercase=True, strip_accents=None, binary=True)
X_train = vectorizer.fit_transform(trainX)
X_dev = vectorizer.transform(devX)
X_test = vectorizer.transform(testX)

_,vocabSize=X_train.shape

le = preprocessing.LabelEncoder()
le.fit(trainY)

Y_train=le.transform(trainY)
Y_dev=le.transform(devY)
Y_test=le.transform(testY)

In [ ]:
def mlp():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(vocabSize,)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model

In [ ]:
def train_and_evaluate(model, verbose=1, epochs=10):
    if verbose > 0:
        print (model.summary())

    # stop training early when the val_loss on the dev data stops going down
    early_stopping = EarlyStopping(monitor='val_loss',
        min_delta=0,
        patience=5,
        verbose=0, 
        mode='auto')

    # when training, save the model that has the best val_loss on the dev data
    modelName="mymodel.hdf5"
    checkpoint = ModelCheckpoint(modelName, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

    # train
    model.fit(X_train, Y_train, 
                validation_data=(X_dev, Y_dev),
                epochs=epochs,
                verbose=verbose,
                callbacks=[checkpoint, early_stopping])
    
    # load best trained model (from checkpoint)
    model.load_weights(modelName)

    dev_loss, dev_accuracy = model.evaluate(X_dev, Y_dev, batch_size=128, verbose=verbose)
    if verbose > 0:
        print("Dev Accuracy: %.3f" % dev_accuracy)

    test_loss, test_accuracy = model.evaluate(X_test, Y_test, batch_size=128, verbose=verbose)
    if verbose > 0:
        print("Test Accuracy: %.3f" % test_accuracy)

    return dev_accuracy, test_accuracy

**Q1: Experiment with the network structure that works best for your binary classification dataset.  Explore the following choices:**
- a) number of layers in the MLP
- b) the size of each layer
- c) the activation functions
- d) the use of dropout  <br><br>
**Create 5 different models and execute them below.  One of the models should be logistic regression (i.e., an MLP with no hidden layers).<br><br>**
**Which architecture performs best on the development data?  <br>**

In [ ]:
def get_model1():
    model = Sequential()
    # your model here

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model

model1=get_model1()
dev_accuracy, test_accuracy=train_and_evaluate(model1)

In [ ]:
def get_model2():
    model = Sequential()
    # your model here

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model

model2=get_model2()
dev_accuracy, test_accuracy=train_and_evaluate(model2)

In [ ]:
def get_model3():
    model = Sequential()
    # your model here

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model

model3=get_model3()
dev_accuracy, test_accuracy=train_and_evaluate(model3)

In [ ]:
def get_model4():
    model = Sequential()
    # your model here

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model

model4=get_model4()
dev_accuracy, test_accuracy=train_and_evaluate(model4)

In [ ]:
def get_logreg():
    model = Sequential()
    # your model here

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
        
    return model
    
logreg=get_logreg()
dev_accuracy, test_accuracy=train_and_evaluate(logreg)

We can generate predictions for a given test set with the `predict_classes` function:

In [ ]:
model=logreg
predictions=model.predict_classes(X_test)

**Q2: For the single best model you identified in Q1 above, calculate 95% confidence intervals it makes on the test data.**

**Q3: Unlike logistic/linear regression, neural networks converge to different solutions as a function of their *initialization* (the random choice of the initial values for parameters).  For the best model that's not logistic regression you identified in Q1 above, train the model 10 times and save the accuracies attained on the development data.  Plot the distribution of dev accuracies using [pandas.DataFrame.plot.density](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.density.html).**